# Visualizing data stored in Cloudant DB from Watson IoT Platform and Anomaly Detection by using IBM Watson Studio

Install the spark-sql-cloudant package (library for reading data from Cloudant database using Spark SQL) in your IBM Watson Studio account by executing the following cell, and then restart the kernel. You will use **pixiedust** to import the required packages.

In [ ]:
!pip install --upgrade pixiedust
import pixiedust
# Use play-json version 2.5.9. Latest version is not supported at this time.
pixiedust.installPackage("com.typesafe.play:play-json_2.11:2.5.9")
# Get the matching version sql-cloudant library
pixiedust.installPackage("org.apache.bahir:spark-sql-cloudant_2.11:0")
# To fix PyJavaError
pixiedust.packageManager.uninstallPackage("org.apache.bahir:spark-sql-cloudant_2.11:2.2.1")

**User input required** : Cloudant credentials.

If you have a connection with Cloudant set up for this project, complete the following steps:
1.	Import your Cloudant credentials by clicking on the following cell below to select it
2.	Click **Find and Add Data** 
3.	Select the Connections tab and click on **Insert to code**. A dictionary called *credentials_1* is  added to the cell that contains the Cloudant credentials. If the dictionary has another name, change it to *credentials_1*. 
4.	Run the cell. 

If you don’t have a connection with Cloudant set up, the credentials can be found on IBM Cloud dashboard by completing the following steps:

1.	Go to your Cloudant service on IBM Cloud, 
2.	Select its Service Credentials section on the left 
3.	Click **View Credentials** to view the username and password. 
4.	Update the *username* and *password* variables with Cloudant’s username and password.

In [ ]:
# This empty cell will be populated with your Cloudant credentials if you follow the steps explained above.

In [ ]:
username = credentials_1["username"]
password = credentials_1["password"]

In [ ]:
host = username + '.cloudant.com'

**User input required**: Cloudant database name.

If you are not sure which database contains the data that you want to import, go to your Cloudant service on IBM Cloud and click **Launch** to display the database name. Update the *dbName* variable with the database name, for example
 *iotp_abcdef_default_2018-01-10*.

In [ ]:
dbName = "DBName"

Connect to the Cloudant database that is generated by WIoTP connector for historical data.

The following code connects to Cloudant NoSQL DB and returns an RDD data frame for the stored IoT data. The line `option("jsonstore.rdd.partitions", 4)` is needed only if your Cloudant service plan is *lite* because this plan has an access quota of 5 requests per second. 
Spark might run parallel jobs that might lead to more than 5 requests being made in one second. If this happens, a "too many requests" error is raised. To resolve this error, decrease the value for the *jsonstore.rdd.partitions* option to 2. For paid service plans this line can be commented out.


In [ ]:
cloudantdata=sqlContext.read.format("org.apache.bahir.cloudant").\
option("cloudant.host", host).\
option("cloudant.username", username).\
option("cloudant.password", password).\
option("view","_design/iotp/_view/by-date").\
option("jsonstore.rdd.partitions", 4).\
load(dbName)

Observe the loaded data:

In [ ]:
cloudantdata.show()

All IoT data is located under the value column.

Next, transform this hierarchical data frame into a flat one, and convert the timestamp from string type into a timestamp type.
The function withColumn adds a column named 'ts' to the data frame, and calculates it's content based on timestamp column (string), by using the to_ts function that was defined.
The cache() function of a data frame caches the data frame in memory, this is very useful when data is accessed repeatedly.

The *deviceData* is a temporary view in the Spark Session and can be used for select statements if you want to write raw SQL. 


In [ ]:
import pandas as pd
from pyspark.sql import *
from pyspark.sql.functions import udf, col, asc, desc,to_date, unix_timestamp, weekofyear, countDistinct
from datetime import datetime
from pyspark.sql.types import DateType, TimestampType, IntegerType

In [ ]:
# This function converts the string cell into a timestamp type:
str_to_ts =  udf (lambda d: datetime.strptime(d, "%Y-%m-%dT%H:%M:%S.%fZ"), TimestampType())

sparkDf = cloudantdata.selectExpr("value.deviceId as deviceId", "value.deviceType as deviceType", "value.eventType as eventType" ,  "value.timestamp as timestamp", "value.data.*","value.data.d.oa as oa","value.data.d.ob as ob","value.data.d.og as og")
sparkDf = sparkDf.withColumn('ts', str_to_ts(col('timestamp')))
sparkDf.cache()
sparkDf.createOrReplaceTempView("deviceData")

# show the resulting schema and data 
sparkDf.printSchema()
spark.sql("SELECT * from deviceData").show(10)


### Change the values of deviceId and deviceType for which you want to see visualizations.

In [ ]:
deviceId = 'phone'
deviceType = 'simulator'

Along with device movement data, you can also add acceleration data(ax,ay,az).

## Data visualization and comprehension

### Device Health 

In this section we will see how to learn about the population of IoT devices and answer questions such as: 
1. How many reports each device type had?
2. What is the breakdown of the devices per device type?
3. How many reports have been sent by each device? 
4. How many reports each event type had? 
5. How many devices reported in a given time interval?

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pprint
from collections import Counter
import numpy as np
from matplotlib import dates

Use Spark to prepare the data for visualization, because Spark can support big data processing. When the data is ready for visualization, convert Spark data Frame into Pandas data Frame, because Pandas has good visualization support.

#### How many reports each device type had? 

Setting the *deviceType* as index of the created Pandas data frame causes the bar plot to be aggregated by the deviceType. Call the plot function of the Pandas data frame.

In [ ]:
EperDtDF = spark.sql("SELECT ts,deviceType from deviceData").groupBy("deviceType").count()
EperDtDF.cache()
EperDtDF.show()

EperDtPanda = EperDtDF.toPandas().set_index('deviceType')

ax = EperDtPanda.plot(kind='bar',legend=False)
ax.set_xlabel("deviceType")
ax.set_ylabel("events count")
ax.set_title('count of events by deviceType')

#### What is the breakdown of the devices per device type? 
The bar chart is plotted in the same way as before, but now we will also show the pie chart of the data. Pandas data frame supports different plot types. Using pie generates a pie chart with percentage sizes of the pieces.
To write the actual count of the devices, instead of percentages, use the autopct argument - multiply by the total amount of devices and divide by 100 to get the actual numbers. 
The total is calculated by using the *sum()* function of Pandas data frame, which sums up the device count of all the deviceTypes. The sum function returns a DataFrame, so use the *[0]* index to get only the value into the total.

In [ ]:
DperDtDF = spark.sql("SELECT deviceId,deviceType from deviceData").groupBy("deviceType").agg(countDistinct('deviceId'))
EperDtDF.cache()
DperDtDF.show()

# bar chart of deviceId by deviceType
EperDtPanda = DperDtDF.toPandas().set_index('deviceType')

ax = EperDtPanda.plot(kind='bar',legend=False)
ax.set_xlabel("deviceType")
ax.set_ylabel("devices count")
ax.set_title('count of deviceIds by deviceType')


# Pie chart of deviceId by deviceType
fig = plt.figure(figsize=(5,5))
ax = plt.subplot(111)
total = EperDtPanda.sum()[0]

ax = EperDtPanda.plot(kind='pie', ax=ax, figsize=(5,5), legend=False, shadow=True, subplots=True, autopct=lambda p: '{:.0f}'.format(p * total / 100))
plt.title("count of deviceIds by deviceType")


#### How many reports have been sent by each device? 

In [ ]:
EperDdf = spark.sql("SELECT deviceId,ts from deviceData").groupBy("deviceId").count()####.sort()########
EperDtDF.cache()
EperDdf.show()

EperDPanda = EperDdf.toPandas().set_index('deviceId')

ax = EperDPanda.plot(kind='bar',legend=False)
ax.set_xlabel("deviceId")
ax.set_ylabel("events count")
ax.set_title('count of events by deviceId')


#### How many reports each event type had? 

In [ ]:
EperEtDF = spark.sql("SELECT eventType,ts from deviceData").groupBy("eventType").count()
EperDtDF.cache()
EperEtDF.show()

EperEtPanda = EperEtDF.toPandas().set_index('eventType')

ax = EperEtPanda.plot(kind='bar',legend=False)
ax.set_xlabel("eventType")
ax.set_ylabel("events count")
ax.set_title('count of events by eventType')

#### How many devices reported in a given time interval?

**User input required**: Replace the year, month and day in the following cell to specify `start` and `end` interval.

For example:

`start = datetime(2017, 7, 28, 0, 0, 0)
end = datetime(2017, 7, 28, 23, 59, 59)`

Make sure that the interval contains device events. You can narrow down the time interval as well. Then run the cell.

In [ ]:
# set the time interval of interest
dbDate = dbName.split("_")[3]
dbDate = dbDate.split("-")

# datetime(year, month, day, hours, minutes, seconds)
start = datetime(int(dbDate[0]), int(dbDate[1]), int(dbDate[2]), 0, 0, 0) 
end = datetime(int(dbDate[0]), int(dbDate[1]), int(dbDate[2]), 23, 59, 59)

print(start)
print(end)

First we filter the data by a time interval, then group the resulting dataFrame by *deviceId*, and count the records for each *deviceId*.

In [ ]:
#filter by time interval
deviceMetaData = sparkDf.select('deviceId','deviceType','ts','timestamp','eventType').filter((col('ts')>=start) & (col('ts')<=end))

deviceMetaData.cache()
#deviceMetaData.show()

#how many devices reported in interval
byDevice = deviceMetaData.groupby(['deviceId']).count()
byDevice.cache()

print("Number of events by deviceId in the interval: ")
byDevice.show()
print("total number of devices reported in the interval:", byDevice.count())

Count of rows by time span for a specific device, using the filter function of Spark DataFrame:

In [ ]:
byDevice.filter(byDevice["deviceId"]== deviceId).show() ##also show 5 with lowest counts

Extract all the numeric columns for further analytics; only a subset of the numeric columns are selected for this demonstration:

In [ ]:
#find all numeric columns of the DataFrame
numericCols = [name_dt for name_dt in sparkDf.dtypes if (('double' in name_dt[1]) or ('int' in name_dt[1]) or ('long' in name_dt[1]))] 

#numericCols is a list of pairs (columnName, dataType), here we select only the column name into the allkeys list
allkeys = [x[0] for x in numericCols]
print("all numeric columns", allkeys)

#select only 5 numeric columns for further detailed visualizations
keys = ['oa','ob','og']
print("selected 3 numeric columns", keys)

#### Device type sensor visualization 

Summary of sensor data that is reported by all devices of a device type, for example:

1. What is the Average/Min/Max of all reported sensor values? 
2. Can I see a histogram of a sensor's output?   
3. What is the correlation between two sensors?

#### Average/Min/Max of all reported sensor values

In [ ]:
from pyspark.mllib.stat import Statistics

#showing visualization for device type 

#show summary only for the selected 5 columns, for easier view, since we have too many columns to fit in a row
dfKeysType = sparkDf.select(*keys).filter(sparkDf["deviceType"]==deviceType)
dfKeysType.cache()

dfKeysType.describe().show()


#### Histogram of a device type sensor's output

1.	Use Spark DataFrame to prepare the histogram for each specific sensor (key) (using rdd.flatMap)
2.	Create Pandas DataFrame from the calculated histogram with 2 columns: "bin" and "frequency".
3.	Plot the histogram using Pandas plot function.

In [ ]:
for key in keys:
    histogram = dfKeysType.select(key).rdd.flatMap(lambda x: x).histogram(11)
    
    #print histogram
    pandaDf = pd.DataFrame(list(zip(list(histogram)[0],list(histogram)[1])),columns=['bin','frequency']).set_index('bin')
    ax =pandaDf.plot(kind='bar')
    ax.set_ylabel("frequency")
    ax.set_title('Histogram of ' + key + ' sensor output')

#### Correlation between two sensors

Correlation between two sensors can be plotted using Pandas plot with kind='scatter'. Remember that *dfKeysType* is a Spark DataFrame that includes only our selected 5 columns and is filtered by *deviceType*. You can also filter by *timestamp* to decrease the amount of data that is output.

In [ ]:
key1="oa"
key2="ob"

pandaDF = dfKeysType.toPandas()
ax = pandaDF.plot(kind='scatter', x=key1, y=key2, s=5, figsize=(7,7))
ax.set_title('Relationship between ' + key1 + ' and ' + key2 )

To view all the correlations of the selected 5 columns, together with a histogram on a diagonal, use the Pandas scatter_matrix function:

In [ ]:
pd.plotting.scatter_matrix(pandaDF, figsize=(18,12))
plt.show()

A correlation matrix can be plotted, using Pandas corr() function on the DataFrame:

In [ ]:
correlations = pandaDF.corr()

# plot correlation matrix
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111)
cax = ax.matshow(correlations, vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,5,1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(keys)
ax.set_yticklabels(keys)
plt.show()

## Sensor deep dive

Sensor deep dive output is similar to the device type sensor visualization but the data is filtered by *deviceId*.

#### Average/Min/Max of all reported sensor values by the device

In [ ]:
#showing visualization for specific deviceID set above
#show summary only for a selected group of columns, for easier view, since we have too many columns to fit in a row
dfKeysDev = sparkDf.select(*keys).filter(sparkDf["deviceId"]==deviceId)
dfKeysDev.cache()

dfKeysDev.describe().show()


A box plot is a method for graphically depicting groups of numerical data through their quartiles. The box extends from the lower to upper quartile values of the data, with a line at the median. The whiskers extend from the box to show the range of the data. Beyond the whiskers, data are considered outliers and are plotted as individual points.

A box plot for each devices sensor, produced with the Pandas plot function with kind="box":

In [ ]:
pandaDF = dfKeysDev.toPandas()
pandaDF.plot(kind='box', subplots=True, layout=(10,3), sharex=False, sharey=False, figsize=(25,60))
plt.show()

#### Histogram of a device's sensor output

In [ ]:
for key in keys:
    try:
        #The histogram is built with spark. Only the groupped by bins data will be converted to Pandas DataFrame
        histogram = dfKeysDev.select(key).rdd.flatMap(lambda x: x).histogram(11)

        #print histogram
        pandaDf = pd.DataFrame(zip(list(histogram)[0],list(histogram)[1]),columns=['bin','frequency']).set_index('bin')
        ax = pandaDf.plot(kind='bar')
        ax.set_ylabel("frequency")
        ax.set_title('Histogram of ' + key + ' sensor output')
   
    except: 
        print("no values for sensor " + key + " for device " + deviceId + "\n")

The histograms can also be built more easily with Pandas DataFrame, in case the dfKeysDev DataFrame is not too large. For the case of big data, spark is more scalable.

In [ ]:
pandaDF = dfKeysDev.toPandas()
pandaDF.hist(layout=(3,3), sharex=False, figsize=(20,15))

#### Density Plots

Kernel density estimation (KDE) is a non-parametric way to estimate the probability density function of a random variable. Kernel density estimation is a fundamental data smoothing problem where inferences about the population are made, based on a finite data sample.

**Note**: here we convert the data into Pandas DataFrame, after we filtered by deviceId and selected a subset of keys. In case this is still too much data for the Pandas DataFrame to handle, consider selecting fewer keys and filtering by time interval.

In [ ]:
pandaDF = dfKeysDev.toPandas()

ax = pandaDF.plot(kind='density', subplots=True, layout=(3,3), sharex=False, figsize=(20,15))
plt.show()

#### How a specific device sensor value changes over time

Maximum, minimum, and average lines are shown on plots.

In [ ]:
from pyspark.sql.functions import mean, min, max

#showing visualization for specific deviceID set above

print(keys)
for key in keys:
    df = spark.sql("SELECT deviceId, ts," + key +" from deviceData where deviceId='" + deviceId + "'").where(col(key).isNotNull())
    df.cache()
    if (df.count() > 0):
        pandaDF = df.toPandas()
        
        ax = pandaDF.plot(x='ts', y=key , legend=False, figsize=(15,9), ls='-', marker='o')
        ax.xaxis.set_major_formatter(dates.DateFormatter('%d-%m-%Y %H:%M:%S'))
        ax.set_title(key + ' over time')
        ax.set_ylabel(key)
        ax.grid(True)
        
        # Draw lines to showcase the upper and lower threshold
        ax.axhline(y=pandaDF[key].min(),c="red",linewidth=2,zorder=0)
        ax.axhline(y=pandaDF[key].max(),c="red",linewidth=2,zorder=0)
        ax.axhline(y=pandaDF[key].mean(),c="green",linewidth=2,zorder=0, ls='--')
    
        ax.autoscale_view()

**Note**: Data can be aggregated by intervals, for example by seconds/minutes/hours in Spark and displayed as aggregated plots (average, minimum, maximum).

In [ ]:
from pyspark.sql.functions import window
from functools import reduce

#showing visualization for specific deviceID set above

for key in keys:
    df = spark.sql("SELECT deviceId, ts," + key +" from deviceData where deviceId='" + deviceId + "'").where(col(key).isNotNull())

    df = df.groupBy("deviceId", window("ts", "30 seconds")).agg(max(key), min(key), mean(key))
    #df = df.groupBy("deviceId", window("ts", "1 minute")).agg(max(key), min(key), mean(key))
    #df.groupBy("deviceId", window("ts", "5 minutes")).agg(max(key), min(key), mean(key))
    #df.groupBy("deviceId", window("ts", "1 hour")).agg(max(key), min(key), mean(key))
    
    #change automatic name of aggregated column
    oldColumns = df.schema.names
    newColumns = ["deviceId", "window", "max", "min", "avg"]
    df = reduce(lambda df, idx: df.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), df)
    
    win_to_ts =  udf (lambda d: d.start, TimestampType())

    df = df.withColumn('ts', win_to_ts(col('window')))
    df = df.select('deviceId','ts',"max", "min", "avg")
    df.cache()
    
    if (df.count() > 0):
        pandaDF = df.toPandas()

        ax = pandaDF.plot(x='ts', y='min', legend=True, figsize=(15,9), ls='-', marker='o', c="red")
        ax = pandaDF.plot(ax=ax, x='ts', y='max', legend=True, figsize=(15,9), ls='-', marker='o', c="red")
        ax = pandaDF.plot(ax=ax, x='ts', y='avg', legend=True, figsize=(15,9), ls='-', marker='o', c="green")
        
        #'S' secondly frequency, 'T' minutely frequency, 'H' hourly frequency
        xtick = pd.date_range(start=pandaDF['ts'].min(), end=pandaDF['ts'].max(), freq='30S')
        ax.set_xticks(xtick)

        ax.xaxis_date()
        ax.set_title(key + ' over time groupped by 30 sec')
        ax.set_ylabel(key)
    
        ax.autoscale_view()

For Big Data is also possible to show the plot around the extremum point. In the example below we show values spanned in 4 seconds around the maximum value. Note that the aggregation is done inside Spark and Pandas is used for visualization.

In [ ]:
from pyspark.sql.functions import mean, min, max

#showing visualization for specific deviceID

for key in keys:
    df = spark.sql("SELECT deviceId, ts," + key +" from deviceData where deviceId='" + deviceId + "'").where(col(key).isNotNull())
    df.cache()
    if (df.count() > 0):
        #find max and filter around them
        max_value = df.select(max(key)).collect()[0][0]
        maxts = df.filter(df[key] == max_value).rdd.map(lambda r: r['ts']).collect()[0]
        startts = maxts - pd.DateOffset(seconds=4) #(minutes=2)#(days=15)
        endts = maxts + pd.DateOffset(seconds=4)
        df_max = df.filter((col('ts')>=startts) & (col('ts')<=endts))

        pandaDF = df_max.toPandas()

        ax = pandaDF.plot(x='ts', y=key , legend=False, figsize=(15,9), ls='-', marker='o')
        ax.xaxis.set_major_formatter(dates.DateFormatter('%d-%m-%Y %H:%M:%S'))
        ax.set_title(key + ' over time')
        ax.set_ylabel(key)
        ax.grid(True)
        
        # Draw lines to showcase the upper and lower threshold
        ax.axhline(y=pandaDF[key].min(),c="red",linewidth=2,zorder=0)
        ax.axhline(y=pandaDF[key].max(),c="red",linewidth=2,zorder=0)
        ax.axhline(y=pandaDF[key].mean(),c="green",linewidth=2,zorder=0, ls='--')
    
        ax.autoscale_view()

#### Compare between the sensor values of devices over time

The *dfKeysDev* DataFrame contains only keys columns, with no ts column, so we will create a new data frame that will also include the *ts*:

In [ ]:
#showing visualization for specific deviceID

columns = list(keys)
columns.append('ts')
df = sparkDf.select(*columns).filter(sparkDf["deviceId"]==deviceId)

pandaDF = df.toPandas().set_index('ts')
ax = pandaDF.plot(figsize=(15,9),ls='', marker='o')   
ax.xaxis.set_major_formatter(dates.DateFormatter('%d-%m-%Y %H:%M:%S'))
ax.set_title(', '.join(keys) + ' over time')
ax.grid(True)
ax.autoscale_view()

In [ ]:
pandaDF = dfKeysDev.toPandas()

pd.plotting.scatter_matrix(pandaDF, figsize=(18,12))
plt.show()

## Anomaly detection


Anomaly detection will be performed using *z-score*. *z-score* is a standard score that indicates how many standard deviations an element is from the mean. A z-score can be calculated from the following formula:
`z = (X - µ) / σ`
where z is the *z-score*, *X* is the value of the element, *µ* is the population mean, and *σ* is the standard deviation.

A higher *z-score* value represents a larger deviation from the mean value which can be interpreted as abnormal.

We will calculate *z-score* for each selected column (sensor) of each device type, and plot only the sensors that have spikes. We define a spike in the following function spike(row), by reported value having z-score above 3 or below -3. Observe that the values for which the *z-score* is above 3 or below -3, are marked as abnormal events in the following graph.

**Note**: The code triggers a number of Spark jobs and might take a few seconds to finish.

In [ ]:
# ignore warnings if any
import warnings
from pyspark.sql.functions import mean, min, max, mean, stddev

warnings.filterwarnings('ignore')

'''
This function detects the spike and dip by returning a non-zero value 
when the z-score is above 3 (spike) and below -3(dip). Incase if you 
want to capture the smaller spikes and dips, lower the zscore value from 
3 to 2 in this function.
'''
upperThreshold = 3
lowerThreshold = -3
def spike(row):
    if(row['zscore'] >=upperThreshold or row['zscore'] <=lowerThreshold):
        return row[key]
    else:
        return 0

#get the list of available devices
deviceTypes = sparkDf.select("deviceType").groupBy("deviceType").count().rdd.map(lambda r: r[0]).collect()

#calculate for each device type and each key
for devt in deviceTypes:
    for key in keys:
        df = spark.sql("SELECT deviceType, ts," + key +" from deviceData where deviceType='" + devt + "'").where(col(key).isNotNull())
        if (df.count() > 0):
            pandaDF = df.toPandas().set_index("ts")
            
            # calculate z-score and populate a new column
            pandaDF['zscore'] = (pandaDF[key] - pandaDF[key].mean())/pandaDF[key].std(ddof=0)

            #add new column - spike, and calculate its value based on the thresholds, usinf spike function, defined above
            pandaDF['spike'] = pandaDF.apply(spike, axis=1)
            
            
            #plot the chart, only if spikes were detected (not all values of "spike" are zero)
            if (pandaDF['spike'].nunique() > 1):
                # select rows that are required for plotting
                plotDF = pandaDF[[key,'spike']]
                #calculate the y minimum value
                y_min = (pandaDF[key].max() - pandaDF[key].min()) / 10
                fig, ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
                ax.set_ylim(plotDF[key].min() - y_min, plotDF[key].max() + y_min)
                x_filt = plotDF.index[plotDF.spike != 0]
                plotDF['spikes'] = plotDF[key]
                y_filt = plotDF.spikes[plotDF.spike != 0]
                #Plot the raw data in blue colour
                line1 = ax.plot(plotDF.index, plotDF[key], '-', color='blue', animated = True, linewidth=1, marker='o')
                #plot the anomalies in red circle
                line2 = ax.plot(x_filt, y_filt, 'ro', color='red', linewidth=2, animated = True)
                #Fill the raw area
                ax.fill_between(plotDF.index, (pandaDF[key].min() - y_min), plotDF[key], interpolate=True, color='blue',alpha=0.6)

                # calculate the sensor value that is corresponding to z-score that defines a spike
                valUpperThreshold = (pandaDF[key].std(ddof=0) * upperThreshold) + pandaDF[key].mean()
                # calculate the sensor value that is corresponding to z-score that defines a dip
                valLowerThreshold = (pandaDF[key].std(ddof=0) * lowerThreshold) + pandaDF[key].mean()

                #plot the thresholds
                ax.axhline(y=valUpperThreshold,c="red",linewidth=2,zorder=0,linestyle='dashed',label='Upper threshold')
                ax.axhline(y=valLowerThreshold,c="red",linewidth=2,zorder=0,linestyle='dotted',label='Lower threshold')
                
                # Label the axis
                ax.set_xlabel("Sequence",fontsize=20)
                ax.set_ylabel(key,fontsize=20)
                ax.set_title("deviceType: " + devt + " sensor:" + key)
                plt.tight_layout()
                plt.legend()
                
                print("Device Type: " + devt + ", sensor: " + key)
                print("Upper treshould based on the z-score calculation: " , upperThreshold , ": " , valUpperThreshold)
                print("Lower treshould based on the z-score calculation: ", lowerThreshold, ": " , valLowerThreshold)
                
                plt.show()

The red marks indicate unexpected spikes where the z-score value is greater than 3 or less than -3. To detect lower spikes, modify the value to 2 or less. Similarly, if you want to detect only the higher spikes, increase the z-score value from 3 to 4 or more.

For complete solution tutorial, refer [Gather, Visualize and Analyze IoT data](http://console.bluemix.net/docs/tutorials/gather-visualize-analyze-iot-data.html#gather-visualize-and-analyze-iot-data)